# Setting enviroment

In [1]:
%reload_ext kedro.ipython

[12/29/24 16:27:25] INFO     Using                                                                  ]8;id=108107;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/framework/project/__init__.py\__init__.py]8;;\:]8;id=431879;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/framework/project/__init__.py#270\270]8;;\
                             '/home/bruno/Documents/Programming/Programming_projects/olist_project/                
                             .venv/lib/python3.11/site-packages/kedro/framework/project/rich_loggin                
                             g.yml' as logging configuration.                                                      

                    INFO     Registered line magic '%reload_kedro'                                   ]8;id=237782;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=511857;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py#61\61]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=787374;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=438560;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py#63\63]8;;\

                    INFO     Resolved project path as:                                              ]8;id=783276;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=887638;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py#178\178]8;;\
                             /home/bruno/Documents/Programming/Programming_projects/olist_project.                 
                             To set a different path, run '%reload_kedro <project_root>'                           

[12/29/24 16:27:28] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=589250;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=924665;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro_telemetry/plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

                    INFO     Kedro project olist_project                                            ]8;id=946728;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=976153;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py#144\144]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=90065;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=807614;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py#145\145]8;;\
                             'pipelines'                                                                           

[12/29/24 16:27:29] INFO     Registered line magic 'run_viz'                                        ]8;id=943794;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=539580;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py#151\151]8;;\

In [2]:
import pandas as pd
import numpy as np
# import shap

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from feature_engine.imputation import CategoricalImputer,ArbitraryNumberImputer
from feature_engine.encoding import RareLabelEncoder, CountFrequencyEncoder
from feature_engine.creation import MathFeatures, RelativeFeatures
from feature_engine.wrappers import SklearnTransformerWrapper

from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from scipy.stats import ks_2samp

import matplotlib.pyplot as plt
import seaborn as sns

import mlflow

import warnings
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import (
    RandomForestClassifier, ExtraTreesClassifier,
    HistGradientBoostingClassifier, GradientBoostingClassifier
)
from sklearn.linear_model import LogisticRegression
from olist_project.utils.model import mlflow_experiment_run_cv
from category_encoders.cat_boost import CatBoostEncoder


In [3]:
pd.set_option('display.max_columns', 110)
pd.set_option('display.max_rows', 110)

In [4]:
mlflow_tracking_uri = context.project_path.as_uri()+'/mlflow'
mlflow.set_tracking_uri(mlflow_tracking_uri)
experiment_name = 'model_selection'
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='file:///home/bruno/Documents/Programming/Programming_projects/olist_project/mlflow/603732696037769194', creation_time=1735500393001, experiment_id='603732696037769194', last_update_time=1735500393001, lifecycle_stage='active', name='model_selection', tags={}>

# Load data

In [10]:
random_state = catalog.load('params:random_state')
id_col = catalog.load('params:audience_building.id_col')
cohort_col = catalog.load('params:audience_building.cohort_col')
target_name = catalog.load('params:modeling.target')
X_dev = catalog.load("X_train")
y_dev = catalog.load("y_train")
y_dev = y_dev[target_name]
id_model_dev = catalog.load("id_model_train")
cohort_dev = pd.to_datetime(id_model_dev[cohort_col], format='%Y%m')

X_oot = catalog.load("X_test_oot")
y_oot = catalog.load("y_test_oot")
y_oot = y_oot[target_name]
id_model_oot = catalog.load("id_model_test_oot")
cohort_oot = pd.to_datetime(id_model_oot[cohort_col], format='%Y%m')

[12/29/24 16:31:55] INFO     Loading data from params:random_state (MemoryDataset)...           ]8;id=914405;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=995102;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

                    INFO     Loading data from params:audience_building.id_col                  ]8;id=607485;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=91042;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from params:audience_building.cohort_col              ]8;id=824177;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=732997;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from params:modeling.target (MemoryDataset)...        ]8;id=843143;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=295279;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

                    INFO     Loading data from X_train (ParquetDataset)...                      ]8;id=677941;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=858895;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

                    INFO     Loading data from y_train (ParquetDataset)...                      ]8;id=40645;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=791436;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

                    INFO     Loading data from id_model_train (ParquetDataset)...               ]8;id=223345;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=325842;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

                    INFO     Loading data from X_test_oot (ParquetDataset)...                   ]8;id=761135;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=275974;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

                    INFO     Loading data from y_test_oot (ParquetDataset)...                   ]8;id=262938;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=904501;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

                    INFO     Loading data from id_model_test_oot (ParquetDataset)...            ]8;id=789103;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=249710;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

# Model definitions

In [6]:
cat_vars = [col for col in X_dev.select_dtypes('category').columns]

def _inf_to_nan(X):
    return X.replace([np.inf, -np.inf], np.nan)
inf_to_nan_transformer = FunctionTransformer(_inf_to_nan)

def get_lgbm_model(X_dev, params=None):
    cat_vars = [col for col in X_dev.select_dtypes('category').columns]
    pipe_steps = []
    if cat_vars:
        pipe_steps.append(('imputer',CategoricalImputer()))
        pipe_steps.append(('encoder',RareLabelEncoder()))
    
    if not params:
        params = dict()
    pipe_steps.append(('estimator',LGBMClassifier(verbosity=-1, random_state=random_state,
                                                  **params)))

    model = Pipeline(pipe_steps)

    return model

In [9]:
models = {}
models['lgbm'] = Pipeline([
    ('imputer',CategoricalImputer()),
    ('encoder',RareLabelEncoder()),
    ('estimator',LGBMClassifier(verbosity=-1, random_state=random_state)),
])

models['lgbm_cat_enc'] = Pipeline([
    ('imputer',CategoricalImputer()),
    ('encoder',RareLabelEncoder()),
    ('cat_encoder',CatBoostEncoder()),
    ('estimator',LGBMClassifier(verbosity=-1, random_state=random_state)),
])

models['xgb'] = Pipeline([
    ('inf_to_nan_transformer',SklearnTransformerWrapper(transformer=inf_to_nan_transformer)),
    ('imputer',CategoricalImputer()),
    ('encoder',RareLabelEncoder()),
    ('encoder_freq',CountFrequencyEncoder()),
    ('estimator',XGBClassifier(random_state=random_state)),
])

models['xgb_cat_enc'] = Pipeline([
    ('inf_to_nan_transformer',SklearnTransformerWrapper(transformer=inf_to_nan_transformer)),
    ('imputer',CategoricalImputer()),
    ('encoder',RareLabelEncoder()),
    ('cat_encoder',CatBoostEncoder()),
    ('estimator',XGBClassifier(random_state=random_state)),
])

models['cat'] = Pipeline([
    ('imputer',CategoricalImputer()),
    ('encoder',RareLabelEncoder()),
    ('estimator',CatBoostClassifier(cat_features=cat_vars, verbose=0, random_state=random_state)),
])

In [16]:
models['lgbm_cat_enc'].fit(X_dev,y_dev)
y_proba = models['lgbm_cat_enc'].predict_proba(X_oot)[:,1]
roc_auc_score(y_oot,y_proba)

0.9335920766032932

In [18]:
models['lgbm_cat_enc'][:-1].transform(X_oot)

,ord_total_orders_m3,ord_nunique_cohorts_m3,ord_mean_estimated_days_to_order_delivery_m3,ord_std_estimated_days_to_order_delivery_m3,ord_max_estimated_days_to_order_delivery_m3,ord_min_estimated_days_to_order_delivery_m3,ord_mean_days_to_order_approval_m3,ord_std_days_to_order_approval_m3,ord_max_days_to_order_approval_m3,ord_min_days_to_order_approval_m3,ord_mean_days_to_order_posting_m3,ord_std_days_to_order_posting_m3,ord_max_days_to_order_posting_m3,ord_min_days_to_order_posting_m3,ord_mean_days_to_order_delivery_m3,ord_std_days_to_order_delivery_m3,ord_max_days_to_order_delivery_m3,ord_min_days_to_order_delivery_m3,ord_mean_diff_days_actual_estimated_delivery_m3,ord_std_diff_days_actual_estimated_delivery_m3,ord_max_diff_days_actual_estimated_delivery_m3,ord_min_diff_days_actual_estimated_delivery_m3,ord_recency_m3,ord_mean_frequency_m3,ord_rate_nunique_cohorts_m3,ord_slope_nunique_order_id_m3,ord_total_orders_m6,ord_nunique_cohorts_m6,ord_mean_estimated_days_to_order_delivery_m6,ord_std_estimated_days_to_order_delivery_m6,ord_max_estimated_days_to_order_delivery_m6,ord_min_estimated_days_to_order_delivery_m6,ord_mean_days_to_order_approval_m6,ord_std_days_to_order_approval_m6,ord_max_days_to_order_approval_m6,ord_min_days_to_order_approval_m6,ord_mean_days_to_order_posting_m6,ord_std_days_to_order_posting_m6,ord_max_days_to_order_posting_m6,ord_min_days_to_order_posting_m6,ord_mean_days_to_order_delivery_m6,ord_std_days_to_order_delivery_m6,ord_max_days_to_order_delivery_m6,ord_min_days_to_order_delivery_m6,ord_mean_diff_days_actual_estimated_delivery_m6,ord_std_diff_days_actual_estimated_delivery_m6,ord_max_diff_days_actual_estimated_delivery_m6,ord_min_diff_days_actual_estimated_delivery_m6,ord_recency_m6,ord_mean_frequency_m6,ord_rate_nunique_cohorts_m6,ord_slope_nunique_order_id_m6,ord_total_orders_m9,ord_nunique_cohorts_m9,ord_mean_estimated_days_to_order_delivery_m9,...,pay_max_value_m9,pay_min_value_m9,pay_max_payment_sequential_m9,pay_median_payment_sequential_m9,pay_min_payment_sequential_m9,pay_count_payment_credit_card_m9,pay_mean_installments_credit_card_m9,pay_std_installments_credit_card_m9,pay_max_installments_credit_card_m9,pay_min_installments_credit_card_m9,pay_sum_value_credit_card_m9,pay_mean_value_credit_card_m9,pay_std_value_credit_card_m9,pay_max_value_credit_card_m9,pay_min_value_credit_card_m9,pay_count_payment_not_credit_card_m9,pay_sum_value_not_credit_card_m9,pay_mean_value_not_credit_card_m9,pay_std_value_not_credit_card_m9,pay_max_value_not_credit_card_m9,pay_min_value_not_credit_card_m9,ctm_nunique_customer_unique_id_m3,ctm_nunique_customer_zip_code_prefix_dig_1_m3,ctm_nunique_customer_zip_code_prefix_dig_2_m3,ctm_nunique_customer_zip_code_prefix_dig_3_m3,ctm_nunique_customer_zip_code_prefix_dig_4_m3,ctm_nunique_customer_zip_code_prefix_dig_5_m3,ctm_nunique_customer_state_m3,ctm_nunique_customer_unique_id_m6,ctm_nunique_customer_zip_code_prefix_dig_1_m6,ctm_nunique_customer_zip_code_prefix_dig_2_m6,ctm_nunique_customer_zip_code_prefix_dig_3_m6,ctm_nunique_customer_zip_code_prefix_dig_4_m6,ctm_nunique_customer_zip_code_prefix_dig_5_m6,ctm_nunique_customer_state_m6,ctm_nunique_customer_unique_id_m9,ctm_nunique_customer_zip_code_prefix_dig_1_m9,ctm_nunique_customer_zip_code_prefix_dig_2_m9,ctm_nunique_customer_zip_code_prefix_dig_3_m9,ctm_nunique_customer_zip_code_prefix_dig_4_m9,ctm_nunique_customer_zip_code_prefix_dig_5_m9,ctm_nunique_customer_state_m9,geo_mean_distance_customer_seller_m3,geo_std_distance_customer_seller_m3,geo_max_distance_customer_seller_m3,geo_min_distance_customer_seller_m3,geo_mean_distance_customer_seller_m6,geo_std_distance_customer_seller_m6,geo_max_distance_customer_seller_m6,geo_min_distance_customer_seller_m6,geo_mean_distance_customer_seller_m9,geo_std_distance_customer_seller_m9,geo_max_distance_customer_seller_m9,geo_min_distance_customer_seller_m9,sel_seller_state
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN